## Robustness (beyond adversarial) with INV test

In [1]:
#this cell of code you can comment out once you installed the following tools in your environment
# pip install panda
# pip install allennlp-models

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.
en-core-web-md 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.3.2 which is incompatible.
constituent-treelib 0.0.5 requires spacy==3.4.1, but you have spacy 3.3.2 which is incompatible.


  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307237 sha256=f6b9af23df6e61b4e186ef2a9fcd95f1dfe140faae6ed75422da121f9f284724
  Stored in directory: c:\users\anaverageone\appdata\local\pip\cache\wheels\59\b9\f8\08050f4b5734886ee64f25d1725e9c1e0a2bd19482b0385f25
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=a245ce76aa740cadc67ba709cece70e7e4ec95f59f776fe605b96745f81e2767
  Stored in directory: c:\users\anaverageone\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=51187d81477bf77ffc4fbc87f07ca47d386c4283d51404129d891052b2291510
  Stored in directory: c:\users\anaverageone\appdata\local\pip\cache\wheels\e1\e8\83\ddea66100678d139b14bc87692ece57c6a2a937956d2532608
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5583 sha256=4f646236f9c0df51079047f8568951b4bc3e82

In [2]:
from sklearn.metrics import classification_report, confusion_matrix
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
from allennlp_models import pretrained
import pandas as pd
import json
import ast
import csv
import os
os.chdir(r'C:\Users\anaverageone\htlt_env\Advanced_NLP')

D:\ANACONDA\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Loading BERT_based and BiLSTM models from AllenNLP tool package
Bert_based = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")
BiLSTM = pretrained.load_predictor("structured-prediction-srl")

error loading _jsonnet (this is expected on Windows), treating C:\Users\ANAVER~1\AppData\Local\Temp\tmpzqw7b56g\config.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating D:\ANACONDA\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json
error loading _jsonnet (this is expected on Windows)

In [4]:
# create Concatenative Adversaries based on examples from en_ewt-up-train.conllu & en_ewt-up-test.conllu 
# 3 x Grammatical Sentences [structurally similar to answer + random sentence];
# 2 x Gibberish [Append distracting text + Meaning-altering typos but does change the lables] 

sentences = [
    "What if POSTNL turned into DPD?", 
    "POSTNL is a nice delivery company.",
    "Why would anybody else use it for anything else?",
    "Was this the jar opener?",
    "I get bloger, of course."
]

gold = [
    ["O", "O", "B-ARG1", "B-V", "O", "B-ARG2", "O"],
    ["B-ARG1", "B-V", "O", "O", "O", "B-ARG2", "O"],
    ["O", "O", "B-ARG0","I-ARG0", "B-V", "B-ARG1", "O", "B-ARG2", "I-ARG2", "O"],
    ["B-V", "B-ARG1", "O", "O", "B-ARG2", "O"],
    ["B-ARG0", "B-V", "B-ARG1", "O", "B-ARGM-ADV", "O", "O"]]

In [5]:
# a function that saves challenge data as csv file 
def create_challenge_data(outfile, test_samples, test_gold):
    with open(outfile, mode='w', newline='') as file:
        writer = csv.writer(file)
    # write headers
        writer.writerow(["Sentence", "Gold"])
    # write each sentence and its corresponding gold labels
        for i in range(len(test_samples)):
            writer.writerow([test_samples[i], test_gold[i]])
            
# a function that prints out the failure rate of model on corresponding challenge data set 
def get_Pred_results(outfile, predictor_name, test_samples):
    with open(outfile, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
    # write headers
        writer.writerow(["Sentence", "Arguments"])
    # Loop through sentences and arguements
        for sent in test_samples:
            output = predictor_name.predict(sentence=sent)
        # for sentences with more than one predicate
            if len(output["verbs"]) > 1:
                arguments = output["verbs"][1]["tags"]
                print(arguments)
                writer.writerow([sent, arguments])
        # for sentences with one predicte 
            elif len(output["verbs"]) == 1:
                arguments = output["verbs"][0]["tags"]
                print(arguments)
                writer.writerow([sent, arguments])
        # for sentences without verbs
            else:
                writer.writerow([sent, ""])
                print('NaN')

# a function that iterate through each token and find matches(both span & lable count as correct)
def calculate_failure_rate(model_name, test_name, gold_arg, model_arg):
    matches_counter = 0
    token_counter = 0
    for i, sent_arg in enumerate(gold_arg):
        gold_list = ast.literal_eval(sent_arg)
        pred_list = ast.literal_eval(model_arg[i])
        for i in range(len(gold_list)):
            if gold_list[i] != pred_list[i]:
                token_counter += 1
            else:
                token_counter += 1
                matches_counter += 1
    failure_rate = (token_counter - matches_counter) / token_counter
    print(f'{model_name} model yeilds a {failure_rate} failure rate for {test_name} test!')

In [6]:
# call "create_challenge_data" to save challenge data as csv file(s)
create_challenge_data('data/robust_file.csv', sentences, gold)

In [7]:
# call "get_Pred_results" to save model predictions as csv file(s)
get_Pred_results('data/Bertbased_robust.csv', Bert_based, sentences)
print()
get_Pred_results('data/BiLSTM_robust.csv', BiLSTM, sentences)

['O', 'O', 'B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'O']
['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['B-ARGM-PRP', 'B-ARGM-MOD', 'B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['O', 'B-V', 'B-ARG2', 'O', 'B-ARGM-ADV', 'I-ARGM-ADV', 'O']

['O', 'O', 'B-ARG1', 'B-V', 'B-ARGM-DIR', 'I-ARGM-DIR', 'O']
['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['B-ARGM-CAU', 'B-ARGM-MOD', 'B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']
['B-ARG0', 'B-V', 'B-ARG1', 'O', 'B-ARGM-DIS', 'I-ARGM-DIS', 'O']


### Evaluation (failure rate) on challenge data (token level)

In [8]:
# Read the test data file and model predictions as pandas dataframe
df_gold = pd.read_csv('data/robust_file.csv', sep=',')
df_bert = pd.read_csv('data/Bertbased_robust.csv', sep=',')
df_bilstm = pd.read_csv('data/BiLSTM_robust.csv', sep=',')
# prepare argument labels for comparison 
bert_arg = df_bert['Arguments'].tolist()
bilstm_arg = df_bilstm['Arguments'].tolist()
gold_arg = df_gold['Gold'].tolist()

### Bert-based model

In [9]:
# call the caculation function to compute failure rate for each model
calculate_failure_rate('Bertbased', 'Robustness', gold_arg, bert_arg)

Bertbased model yeilds a 0.43243243243243246 failure rate for Robustness test!


### BiLSTM 

In [10]:
calculate_failure_rate('BiLSTM', 'Robustness', gold_arg, bilstm_arg)

BiLSTM model yeilds a 0.40540540540540543 failure rate for Robustness test!
